In [8]:
import os
from glob import glob
import shutil
import json
from tqdm import tqdm
import pandas as pd
from pathlib import Path
import jsonlines
import hashlib
import re
from PIL import Image
src = "/home/kai/workspace/DeepDocs_Project/datalake/source/dataset=post_handwritten_plain_text"

In [15]:
images = glob(os.path.join(src, "images", "*.*g"))
jsonl_path = glob(os.path.join(src, "*.jsonl"))[0]

In [16]:
jsonl_path

'/home/kai/workspace/DeepDocs_Project/datalake/source/dataset=post_handwritten_plain_text/metadata.jsonl'

In [17]:
with jsonlines.open(jsonl_path, mode='r') as reader:
    data = [obj for obj in reader]

In [55]:
import numpy as np
from PIL import Image
def get_safe_image_hash(path):
    img = Image.open(path).convert("RGB")
    arr = np.array(img)
    meta = f"{arr.shape}{arr.dtype}".encode()
    return hashlib.sha256(arr.tobytes() + meta).hexdigest()

def get_safe_image_hash_from_pil(img: Image.Image) -> str:
    arr = np.array(img.convert("RGB"))
    meta = f"{arr.shape}{arr.dtype}".encode()
    return hashlib.sha256(arr.tobytes() + meta).hexdigest()

In [69]:
records = []
MAX_SIZE = 2048
os.makedirs("images", exist_ok=True)
for d_ in tqdm(data):
    file_name = d_['file_name']
    gts = json.loads(d_['ground_truth'])['gt_parse']['주소 정보']
    label = {"address": []}
    for gt_ in gts:
        gt = gt_[list(gt_.keys())[0]]
        text = gt['도로명 주소'].strip()
        label['address'].append({
            "<|value|>": text,
        })
    if not os.path.exists(os.path.join(src, "images", file_name)):
        print(f"Image {file_name} not found in source directory.")
        continue
    image = Image.open(os.path.join(src, "images", file_name))
    
    width, height = image.size
    scale = min(MAX_SIZE / width, MAX_SIZE / height)
    if scale < 1:
        new_size = (int(width * scale), int(height * scale))
        image = image.resize(new_size, Image.LANCZOS)
    width, height = image.size
    hash = get_safe_image_hash_from_pil(image)
    image_basename = hash + ".jpg"
    image.save(os.path.join("images", image_basename), quality=95, subsampling=0)
    records.append({
        "image_path": image_basename,
        "width": width,
        "height": height,
        "label": label,
    })

  0%|          | 0/507 [00:00<?, ?it/s]

 11%|█         | 57/507 [00:08<00:52,  8.63it/s]

Image 0000.jpg not found in source directory.


100%|██████████| 507/507 [01:33<00:00,  5.42it/s]


In [70]:
len(os.listdir("images"))

506

In [71]:
df = pd.DataFrame(records)

In [72]:
df.to_parquet("data.parquet", index=False)

In [61]:
hash

'255ad80a6f900a3a19f5ebf55b03094e5ec96770a40638114b1e1a894b298989'

In [54]:
records

[{'image_path': '0009.jpg',
  'label': {'address': [{'<|value|>': '옥천군 안내면 용촌2길 12'},
    {'<|value|>': '영암군 도포면 덕화만수로 264'},
    {'<|value|>': '김천시 남면 송곡길 296'},
    {'<|value|>': '공주시 이인면 용화대1길 7'},
    {'<|value|>': '경주시 불국사초등길 20'},
    {'<|value|>': '연제구 법원북로83번길 9'},
    {'<|value|>': '무안군 현경면 고인돌길 207'},
    {'<|value|>': '고흥군 두원면 우주항공로 2333'},
    {'<|value|>': '진도군 조도면 맹골도길 13'},
    {'<|value|>': '진안군 진안읍 연구동길 5'},
    {'<|value|>': '화순군 화순읍 서양로 200'},
    {'<|value|>': '하남시 춘궁로35번길 90'},
    {'<|value|>': '영천시 하명길 12'},
    {'<|value|>': '상주시 외남면 신촌내동길 12'},
    {'<|value|>': '성남시 수정구 고등공원로 19'},
    {'<|value|>': '부강면 행산길 8'},
    {'<|value|>': '원주시 호저면 막골길 70'},
    {'<|value|>': '청양군 운곡면 방축길 76'},
    {'<|value|>': '당진시 정미면 모평제방길 45'},
    {'<|value|>': '안동시 남선면 양지길 14'}]}},
 {'image_path': '0010.jpg',
  'label': {'address': [{'<|value|>': '청양군 운곡면 유의각길 98'},
    {'<|value|>': '봉화군 법전면 호산길 203'},
    {'<|value|>': '동해시 묘골길 24'},
    {'<|value|>': '안산시 단원구 목내로119번길 74'},
 

In [52]:
text

'안동시 남선면 양지길 14'

In [37]:
gt

[{'0': {'우편번호': '', '도로명 주소': ' 옥천군 안내면 용촌2길 12', '지번 주소': '', '건물명': ''}},
 {'1': {'우편번호': '', '도로명 주소': ' 영암군 도포면 덕화만수로 264', '지번 주소': '', '건물명': ''}},
 {'2': {'우편번호': '', '도로명 주소': ' 김천시 남면 송곡길 296', '지번 주소': '', '건물명': ''}},
 {'3': {'우편번호': '', '도로명 주소': ' 공주시 이인면 용화대1길 7', '지번 주소': '', '건물명': ''}},
 {'4': {'우편번호': '', '도로명 주소': ' 경주시 불국사초등길 20', '지번 주소': '', '건물명': ''}},
 {'5': {'우편번호': '', '도로명 주소': ' 연제구 법원북로83번길 9', '지번 주소': '', '건물명': ''}},
 {'6': {'우편번호': '', '도로명 주소': ' 무안군 현경면 고인돌길 207', '지번 주소': '', '건물명': ''}},
 {'7': {'우편번호': '', '도로명 주소': ' 고흥군 두원면 우주항공로 2333', '지번 주소': '', '건물명': ''}},
 {'8': {'우편번호': '', '도로명 주소': ' 진도군 조도면 맹골도길 13', '지번 주소': '', '건물명': ''}},
 {'9': {'우편번호': '', '도로명 주소': ' 진안군 진안읍 연구동길 5', '지번 주소': '', '건물명': ''}},
 {'10': {'우편번호': '', '도로명 주소': ' 화순군 화순읍 서양로 200', '지번 주소': '', '건물명': ''}},
 {'11': {'우편번호': '', '도로명 주소': ' 하남시 춘궁로35번길 90', '지번 주소': '', '건물명': ''}},
 {'12': {'우편번호': '', '도로명 주소': ' 영천시 하명길 12', '지번 주소': '', '건물명': ''}},
 {'13': {'우편번호